In [4]:
pwd

'/content'

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/Chat Bot

/content/drive/MyDrive/Colab Notebooks/Chat Bot


In [6]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Chat Bot'

In [7]:
# libraries for NLP
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

# libraries for tensorflow
import tensorflow as tf
from tensorflow import keras
import numpy as np
import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
# import chatbot intent files
with open('intents.json') as jsondata:
  intents = json.load(jsondata)

In [9]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context_set': ''},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'chatbot',
   'patterns': ['Who built this chatbot?',
    'Tell me about Chatbot',
    'What is this chatbot name?'],
   'responses': ['Hi, I am Chatbot designed by Bilal.',
    'Thanks for asking. I am designed by Bilal Haneef.',
    'I am a chatbot.']},
  {'tag': 'location',
   'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?'],
   'responses': ['We are from Pakis

## Exploring intents

In [10]:
((intents['intents'])[0])

{'tag': 'greeting',
 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
 'responses': ['Hello, thanks for visiting',
  'Good to see you again',
  'Hi there, how can I help?'],
 'context_set': ''}

In [11]:
((intents['intents'])[0])['responses']


['Hello, thanks for visiting',
 'Good to see you again',
 'Hi there, how can I help?']

In [12]:
for intent in intents['intents']:
  for ptrn in intent['patterns']:
    print(ptrn)

Hi
How are you
Is anyone there?
Hello
Good day
Bye
See you later
Goodbye
Thanks
Thank you
That's helpful
Who built this chatbot?
Tell me about Chatbot
What is this chatbot name?
What is your location?
Where are you located?
What is your address?
Give me your social media accounts link
Where can we connect
How can i reach out to you?
Is there any way we can connect
Which is your favourite movie?
Suggest me some movies
Recommend movies
Who are you?
Tell me about Yourself
What is this?


In [13]:
for intent in intents['intents']:
  # for tag in intent['tag']:
    print(intent['tag'])

greeting
goodbye
thanks
chatbot
location
connect
movies
about


In [14]:
words = []
classes = []
documents = []
ignore = ['?'] # if we want to remove special character we can put here

for intent in intents['intents']:
  for ptrn in intent['patterns']:
    # tokenize every word in the sentence
    token = nltk.word_tokenize(ptrn)
    # print(token)
    # adding word to words list
    words.extend(token) # using append is not useful as it would have added the whole sentence as a list in the words, we want to push each word separatelya
    # adding words along with tag to documents list
    documents.append((token,intent['tag']))
    # adding tags to classes list
    if intent['tag'] not in classes:  # we want to append only unique tags
      classes.append(intent['tag'])

## **stemmer.stem()** function is used to perform stemming on words. Stemming is the process of reducing words to their base or root form, known as the "stem".

### PorterStemmer from NLTK is used to stem the word "running." The stemmer.stem(word) function applies the stemming algorithm to the word, resulting in the base form "run.

In [15]:
# Example
stemmer = PorterStemmer()
test = ["cats", "playing", "jumped",'achieved','fighting']
test = [stemmer.stem(word) for word in test]
print(test)

['cat', 'play', 'jump', 'achiev', 'fight']


In [16]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words))) # converting words to set so that repeating words gets removed and then again convert it into list from set then sort

In [17]:
classes = sorted(list(set(classes))) # convertint to set just to make sure that there are no repeating words

In [18]:
print(len(words))
print(len(classes))
print(len(documents))

52
8
27


In [19]:
# creating training data
x = []
y = []

# creating an empty array for the output
outempty = [0] * len(classes) # [0] will be multiplied by # of classes available , [0] * 4 = [0 0 0 0]

# creating training set, bag of words for each sentence
for pair in documents:
  # instantiating bag of words
  bag = []

  # extracting sentence from pair containing (sentence,tag)
  pattern = pair[0]

  # converting the words in pattern to their base form
  pattern = [stemmer.stem(word.lower()) for word in pattern]

  # checking if words in pattern available in 'words(variable)', if yes then append 1 in bag else 0
  for w in words:
    bag.append(1) if w in pattern else bag.append(0)

  # changing outempty to list, it was already a list but just to make sure.
  outempty_updated = list(outempty)

  # assigning index places to 1 of outempty array wherever the tag is positioned in classes array
  outempty_updated[classes.index(pair[1])] = 1

  x.append([bag,outempty_updated])

random.shuffle(x)
x = np.array(x)
# xtrain = list(x[:,0])
# ytrain = list(x[:,1])
xtrain = list(x[:,0]) # contaiting words as x to train model
ytrain = list(x[:,1]) # containing classes as y to train model

<ipython-input-19-fffa11de991c>:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x = np.array(x)


In [22]:
xtrain[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
ytrain

[[0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0]]

# **Model Training**

In [ ]:
len(xtrain[0]) , len(ytrain[0])

(52, 8)

In [ ]:
xtrain = np.array(xtrain)
ytrain = np.array(ytrain)

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(10,input_shape = (len(xtrain[0]),)),
    keras.layers.Dense(30,activation = 'relu'),
    keras.layers.Dense(30,activation = 'relu'),
    keras.layers.Dense(10,activation = 'relu'),
    keras.layers.Dense(len(ytrain[0]),activation = 'softmax')
])
model.compile(optimizer = 'adam',loss ='categorical_crossentropy',metrics = 'accuracy')
model.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 2.0860 - accuracy: 0.0741
Epoch 2/100
1/1 [==============================] - 0s 12ms/step - loss: 2.0792 - accuracy: 0.0741
Epoch 3/100
1/1 [==============================] - 0s 10ms/step - loss: 2.0727 - accuracy: 0.1111
Epoch 4/100
1/1 [==============================] - 0s 11ms/step - loss: 2.0664 - accuracy: 0.1481
Epoch 5/100
1/1 [==============================] - 0s 10ms/step - loss: 2.0603 - accuracy: 0.1481
Epoch 6/100
1/1 [==============================] - 0s 9ms/step - loss: 2.0541 - accuracy: 0.1852
Epoch 7/100
1/1 [==============================] - 0s 12ms/step - loss: 2.0481 - accuracy: 0.1852
Epoch 8/100
1/1 [==============================] - 0s 13ms/step - loss: 2.0420 - accuracy: 0.1852
Epoch 9/100
1/1 [==============================] - 0s 10ms/step - loss: 2.0359 - accuracy: 0.1852
Epoch 10/100
1/1 [==============================] - 0s 9ms/step - loss: 2.0298 - accuracy: 0.2222
Epoch 11/100
1/1 [=====

In [ ]:
model.save('chatbot.pkl')